In [ ]:
!pip install ultralytics easyocr paddleocr paddlepaddle matplotlib langchain fastapi opencv-python



In [ ]:
! pip install --upgrade

# Core packages
! pip install numpy==1.24.3  # Compatible with most
! pip install opencv-python==4.12.0.88 opencv-contrib-python==4.12.0.88

# YOLO
! pip install ultralytics==8.3.227

# OCR
! pip install easyocr==1.7.2 paddleocr==2.4 paddlepaddle==3.2.2

# Others
! pip install matplotlib fastapi


ERROR: You must give at least one requirement to install (see "pip help install")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 20.1 MB/s  0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
ERROR: Failed to build 'numpy' when getting requirements to build wheel
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 MB 38.2 MB/s  0:00:01
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.10.0.84
    Uninstalling opencv-contrib-python-4.10.0.84:
      Successfully uninstalled opencv-contrib-python-4.10.0.84
  Using cached ultralytics-8.3.227-py3-none-any.whl.metadata (37 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.7 MB/s  0:0

In [ ]:
# ============================================================
# FULL LICENSE PLATE DETECTOR (COLAB READY, NO PADDLEOCR)
# ============================================================

import cv2
import re
import json
import os
import sqlite3
import numpy as np
from datetime import datetime
from google.colab.patches import cv2_imshow
from google.colab import files
from ultralytics import YOLO
import easyocr


# ============================================================
# Initialize EASYOCR
# ============================================================
ocr = easyocr.Reader(['en'])


# ============================================================
# OCR FUNCTION (EasyOCR)
# ============================================================
def easyocr_plate(frame, x1, y1, x2, y2):
    roi = frame[y1:y2, x1:x2]

    if roi.size == 0:
        return ""

    results = ocr.readtext(roi)

    extracted_texts = []
    for item in results:
        text = item[1]
        extracted_texts.append(text)

    final_text = "".join(extracted_texts)
    final_text = re.sub(r'\W+', '', final_text)
    final_text = final_text.replace("O", "0")

    return final_text


# ============================================================
# SAVE JSON + DB
# ============================================================
def save_json(license_plate, starttime, endtime):
    interval_data = {
        'starttime': starttime.isoformat(),
        'endtime': endtime.isoformat(),
        'license_plate': list(license_plate)
    }

    os.makedirs('/content/json', exist_ok=True)

    interval_path = f'/content/json/output_{datetime.now().isoformat()}.json'
    with open(interval_path, 'w') as f:
        json.dump(interval_data, f, indent=2)

    master_path = '/content/json/license_plate_data.json'
    if os.path.exists(master_path):
        with open(master_path, 'r') as f:
            existing = json.load(f)
    else:
        existing = []

    existing.append(interval_data)

    with open(master_path, 'w') as f:
        json.dump(existing, f, indent=2)

    save_json_dbbase(license_plate, starttime, endtime)


def save_json_dbbase(license_plate, starttime, endtime):
    conn = sqlite3.connect('/content/license_plate.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            starttime TEXT,
            endtime TEXT,
            license_plate TEXT
        )
    ''')

    for plate in license_plate:
        cursor.execute('''
            INSERT INTO history (starttime, endtime, license_plate)
            VALUES (?, ?, ?)
        ''', (starttime.isoformat(), endtime.isoformat(), plate))

    conn.commit()
    conn.close()


# ============================================================
# MAIN DETECTOR CLASS
# ============================================================
class LicensePlateDetector:
    def __init__(self, video_path, model_path, conf_threshold=0.5, save_interval=5, display=False):
        self.video_path = video_path
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        self.save_interval = save_interval
        self.license_plate_set = set()
        self.start_time = datetime.now()
        self.display = display
        self.frame_count = 0

    # --------------------------------------
    def process_frame(self, frame):
        results = self.model.predict(frame, verbose=False)

        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            confs = result.boxes.conf.cpu().numpy()

            for (x1, y1, x2, y2), conf in zip(boxes, confs):
                if conf < self.conf_threshold:
                    continue

                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"LP {conf:.2f}", (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                label = easyocr_plate(frame, x1, y1, x2, y2)
                if label:
                    self.license_plate_set.add(label)

                    cv2.putText(frame, label, (x1, y1 - 25),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        return frame

    # --------------------------------------
    def save_if_needed(self):
        now = datetime.now()
        if (now - self.start_time).total_seconds() >= self.save_interval:
            save_json(self.license_plate_set, self.start_time, now)
            self.start_time = now
            self.license_plate_set.clear()

    # --------------------------------------
    def run(self):
        cap = cv2.VideoCapture(self.video_path)

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_path = "/content/output_processed.mp4"
        fps = cap.get(cv2.CAP_PROP_FPS)
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        writer = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            self.frame_count += 1
            print("Processing frame:", self.frame_count)

            frame = self.process_frame(frame)
            writer.write(frame)
            self.save_if_needed()

            if self.display:
                cv2_imshow(frame)

        cap.release()
        writer.release()

        print("Processing complete!")
        print("Downloading video...")
        files.download(output_path)


# ============================================================
# Example Usage
# ============================================================
# detector = LicensePlateDetector(
#     video_path="/content/myvideo.mp4",
#     model_path="/content/best.pt",
#     conf_threshold=0.5,
#     save_interval=5,
#     display=False
# )
# detector.run()


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
detector = LicensePlateDetector(
    video_path="/content/LicensePlateDetectionTest_1080p.mp4",
    model_path="/content/best_licence_patedt15.pt",
    conf_threshold=0.3,
    save_interval=5,
    display=False
)
detector.run()


Processing frame: 1


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Processing frame: 2
Processing frame: 3
Processing frame: 4
Processing frame: 5
Processing frame: 6
Processing frame: 7
Processing frame: 8
Processing frame: 9
Processing frame: 10
Processing frame: 11
Processing frame: 12
Processing frame: 13
Processing frame: 14
Processing frame: 15
Processing frame: 16
Processing frame: 17
Processing frame: 18
Processing frame: 19
Processing frame: 20
Processing frame: 21
Processing frame: 22
Processing frame: 23
Processing frame: 24
Processing frame: 25
Processing frame: 26
Processing frame: 27
Processing frame: 28
Processing frame: 29
Processing frame: 30
Processing frame: 31
Processing frame: 32
Processing frame: 33
Processing frame: 34
Processing frame: 35
Processing frame: 36
Processing frame: 37
Processing frame: 38
Processing frame: 39
Processing frame: 40
Processing frame: 41
Processing frame: 42
Processing frame: 43
Processing frame: 44
Processing frame: 45
Processing frame: 46
Processing frame: 47
Processing frame: 48
Processing frame: 49


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>